In [1]:
import altair as alt
from collections import namedtuple
import pandas as pd
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
def load_data():
    df = pd.read_json("/lab/data/sensex.json")
    df.columns = ['date', 'open', 'high', 'low', 'close', 'adj_close', 'volume']
    df = df[['date', 'close']]
    df = df[~df['close'].isin(['-'])]
    df.date = pd.to_datetime(df.date, format='%b %d, %Y')
    df.close = df.close.str.replace(',', '').astype(float)
    df = df.sort_values(by='date')
    df = df.reset_index(drop=True)
    
    # Resample to weeks
    # df['weeknum'] = df.date.dt.isocalendar().year * 100 + df.date.dt.isocalendar().week
    # df = df.groupby('weeknum')[['date', 'close']].nth(-1).reset_index(drop=True)
    return df

df = load_data()

In [3]:
df

,date,close
0,2003-09-01,4324.76
1,2003-09-02,4339.20
2,2003-09-03,4257.94
3,2003-09-04,4310.51
4,2003-09-05,4369.17
...,...,...
4536,2022-02-07,57621.19
4537,2022-02-08,57808.58
4538,2022-02-09,58465.97
4539,2022-02-10,58926.03


In [6]:
def equity_curve(df, period):
    starting_capital = 1_00_000
    equity_curve = []
    EquityCurve = namedtuple("EquityCurve", ["date", "cash", "units", "price", "ma", "pf_value"])
    
    df['ma'] = df.close.rolling(window=period).mean().round(2)
    df = df.dropna()
    init = df.iloc[0]
    tail = df[1:].reset_index(drop=True)
    
    # calc first entry
    if init.close > init.ma:
        entry = EquityCurve(
            date=init.date,
            cash=0,
            units=round(starting_capital/init.close, 4),
            price=init.close,
            ma=init.ma,
            pf_value=starting_capital
        )
        
    else:
        entry = EquityCurve(
            date=init.date,
            cash=starting_capital,
            units=0,
            price=init.close,
            ma=init.ma,
            pf_value=starting_capital
        )
        
    equity_curve.append(entry)
    
    
    # LOOP THROUGH EVERY DATE
    for index, row in tail.iterrows():
        prev = equity_curve[index]
        is_deployed = prev.units != 0
        
        # update holdings value
        if is_deployed and row.close > row.ma:
            entry = EquityCurve(
                date=row.date,
                cash=0,
                units=prev.units,
                price=row.close,
                ma=row.ma,
                pf_value=round(prev.units * row.close, 2)
            )
        
        # move to cash
        if is_deployed and row.close <= row.ma:
            entry = EquityCurve(
                date=row.date,
                cash=round(prev.units * row.close, 2),
                units=0,
                price=row.close,
                ma=row.ma,
                pf_value=round(prev.units * row.close, 2)
            )

        # deploy cash
        if not is_deployed and row.close > row.ma:
            entry = EquityCurve(
                date=row.date,
                cash=0,
                units=round(prev.pf_value/row.close, 4),
                price=row.close,
                ma=row.ma,
                pf_value=prev.pf_value
            )
        
        # continue with previous day's value
        if not is_deployed and row.close <= row.ma:
            entry = EquityCurve(
                date=row.date,
                cash=prev.cash,
                units=0,
                price=row.close,
                ma=row.ma,
                pf_value=prev.pf_value
            )
        
        equity_curve.append(entry)
    
    return equity_curve

eq_curve = equity_curve(df, 100)
pf = pd.DataFrame(eq_curve)

In [7]:
pf

,date,cash,units,price,ma,pf_value
0,2004-01-22,0.00,17.8771,5593.74,5025.86,100000.00
1,2004-01-23,0.00,17.8771,5816.64,5040.78,103984.65
2,2004-01-27,0.00,17.8771,5993.06,5057.32,107138.53
3,2004-01-28,0.00,17.8771,5876.05,5073.50,105046.73
4,2004-01-29,0.00,17.8771,5802.75,5088.42,103736.34
...,...,...,...,...,...,...
4437,2022-02-07,999372.82,0.0000,57621.19,59197.53,999372.82
4438,2022-02-08,999372.82,0.0000,57808.58,59188.38,999372.82
4439,2022-02-09,999372.82,0.0000,58465.97,59181.63,999372.82
4440,2022-02-10,999372.82,0.0000,58926.03,59180.73,999372.82


In [8]:
pf.price = (pf.price.pct_change() + 1).fillna(100000).cumprod().round(2)
pf = pd.melt(pf, id_vars=['date'], value_vars=['price', 'pf_value'])

pf

,date,variable,value
0,2004-01-22,price,100000.00
1,2004-01-23,price,103984.81
2,2004-01-27,price,107138.69
3,2004-01-28,price,105046.89
4,2004-01-29,price,103736.50
...,...,...,...
8879,2022-02-07,pf_value,999372.82
8880,2022-02-08,pf_value,999372.82
8881,2022-02-09,pf_value,999372.82
8882,2022-02-10,pf_value,999372.82


In [9]:
alt.Chart(pf).mark_line().encode(
    x='date',
    y='value',
    color='variable',
    tooltip=['date', 'value', 'variable']
).properties(
    title="Sensex 100DMA Strategy",
    width=800,
    height=600
)

alt.Chart(...)